In [1]:
import os
import sys

import cv2 as cv 
import numpy as np
from tqdm import tqdm   
from PIL import Image 
from skimage import segmentation

In [2]:
def _get_centroid_1(nucleus_contour):
    '''
    return centroid coordinate of a nucleus,
    manually, average of all coordinates
    
    Parameters
    ----------
    @nucleus_contour: a numpy array of a series of coordinates
    '''
    
    length = len(nucleus_contour)
    sum_x = np.sum(nucleus_contour[:, 0])
    sum_y = np.sum(nucleus_contour[:, 1])
    
    return int(sum_x/length), int(sum_y/length)


def _get_centroid_2(cnt):
    '''
    return centroid coordinate of a nucleus
    use cv moment
    
    Parameters
    ----------
    @cnt: a N * 1 * 2 numpy array, where N is the number of pixel in contour
    '''
    
    M = cv.moments(cnt)
    
    cX = 0
    cY = 0
    
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        
    return cX, cY


def _my_superpixel_segments(img, vor_annos, compactness=30):
    '''
    generate superpixel labels of the given image, only superpixels that contains
    point annotations are  kept.
    '''
    H, W, _ = img.shape
    temp_vor_annos = vor_annos == 1
    

    #get superpixel segmentation
    segments = segmentation.slic(img, compactness=30, start_label=1)
    if np.min(segments) == 0:
        segments += 1

    mask = np.zeros((H, W), dtype=np.uint8)

    #only keep superpixel contains points
    for i in range(np.min(segments), np.max(segments)+1):
        if np.sum((segments==i)*temp_vor_annos) > 0:
            mask += (segments==i)
    
    return np.clip(mask.astype(np.uint8), 0, 1)


def gen_pseudo_labels(img, mask):
    '''
    generate label data for training
    
    Parameters
    ----------
    @img: input image
    @mask: TNBC mask img
    
    Parameters
    ----------
    @A 3 channel image
    @channel 0, store ground truth
    @channel 1, store centroid and voronoi boundary
    @channel 2, store selected pseudo labels
    '''
    #find contours of each cell
    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    cnts = []      
                                                        
    label_channel_0 = np.zeros((256, 256), dtype=np.uint8)     #channel 0, store ground truth
    label_channel_1 = np.zeros((256, 256), dtype=np.uint8)     #channel 1, store centroid and voronoi boundary
    label_channel_2 = np.zeros((256, 256), dtype=np.uint8)     #channel 2, store selected pseudo labels
                                                               
    subdiv = cv.Subdiv2D((0, 0, 256, 256))           #store voronoi diagram

    #draw voronoi boundary  channel_1
    for e in contours:
        ctd = _get_centroid_2(e)

        #print('ctd: ', ctd)
        label_channel_1[ctd[1], ctd[0]] = 1

        #print('ctd: ', ctd)
        subdiv.insert(ctd)    
    
    (facets, centers) = subdiv.getVoronoiFacetList([])
    for e in facets:
        cv.polylines(label_channel_1, [e.astype(np.int)], True, 2)
    
    #draw ground truth    channel_0
    label_channel_0 = np.clip(mask, 0, 1)

    #draw pseudo ground truth  channel_2
    label_channel_2 = _my_superpixel_segments(img, label_channel_1, 'Mo')

    return label_channel_2

In [3]:
def dataset_TNBC():
    '''
    Generate supervision for TNBC dataset, 
    for details, see my_dataset.py
    '''
    image_path = '/home/kunzixie/Medical_Image_Analysis/pytorch-maml/patches/CPM/images/'
    gt_path = '/home/kunzixie/Medical_Image_Analysis/pytorch-maml/patches/CPM/labels/'
    sp_path = '/home/kunzixie/Medical_Image_Analysis/pytorch-maml/sp/CPM_sp'
    #define path
    data_ids = [i[:-4] for i in os.listdir(image_path)]  

    print(f'Generating superpixel...')
    sys.stdout.flush()
   
    
    for i in tqdm(range(len(data_ids)), total=len(data_ids)):
        #read image and ground truth
        ground_truth = cv.imread(os.path.join(gt_path, data_ids[i]+'_label.png'), cv.IMREAD_GRAYSCALE)    
        ground_truth = np.clip(ground_truth, 0, 1)
        img = cv.imread(os.path.join(image_path, data_ids[i]+'.png'))            
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
        #generate
        sp = gen_pseudo_labels(img, ground_truth)
                                                    
      
        #write origin images, labels        
        cv.imwrite(os.path.join(sp_path, data_ids[i]+'.png'), sp*255)    #pseudo
        
    print('Done!')

In [4]:
dataset_TNBC()

Generating superpixel...


  0%|                                                   | 0/256 [00:00<?, ?it/s]/tmp/ipykernel_2788161/3181772385.py:103: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cv.polylines(label_channel_1, [e.astype(np.int)], True, 2)
100%|█████████████████████████████████████████| 256/256 [00:10<00:00, 25.50it/s]

Done!
